<a href="https://colab.research.google.com/github/h3it-dias/FastCamp_Agent_AI_CEIA/blob/main/Pydantic/Pydantic_aula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pydantic
!pip install 'pydantic[email]'

Importação dos modulos, bem como suas ferramentas que serão trabalhadas.

In [16]:
import hashlib
import re
from typing import Any
import enum

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

Estabelecimento do padrão de senha e nome, considerados válidos, em regex.

In [12]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

Criação da classe resposável por enumerar as funções dos usuários (class Role) e da classe User, que cuida do modelo como os dados devem seguir na hora de instanciar o objeto. Um acréscimo com relação ao exemplo Pydantic_aula1, a estrutura de validação personalizada que foi criada, com o field_validator, que estabele uma validação do nome, uma validação da função/role do usário e uma validação do modelo geral - que nesse ultimo caso é muito útil para determinar quando a senha é o nome do usuário.

In [13]:
class Role(enum.IntFlag):
  Author = 1
  Editor = 2
  Admin = 4
  SuperAdmin = 8

class User(BaseModel):
  name: str = Field(examples=["Arjan"])
  email: EmailStr = Field(
      examples=["user@arjancodes.com"],
      description="The email address of the user",
      frozen=True,
  )
  password: SecretStr = Field(
      examples=["Password123"], description="The password of the user"
  )
  role: Role = Field(
      default=None, description="The role of the user", example=[1, 2, 4, 8]
  )

  @field_validator("name")
  @classmethod
  def validate_name(cls, v: str) -> str:
    if not VALID_NAME_REGEX.match(v):
      raise ValueError(
          "Name is invalid, must contain only letters and be at least 2 characters long"
      )
    return v



  @field_validator("role", mode="before")
  @classmethod
  def validate_role(cls, v: int | str |Role) -> Role:
    op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
    try:
      return op[type(v)](v)
    except (KeyError, ValueError):
      raise ValueError(
          f"Role is invalid, please use one of the following: {", ".join([x.name for x in Role])}"
      )

  @model_validator(mode="before")
  @classmethod
  def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
    if "name" not in v or "password" not in v:
      raise ValueError("Name and password are required")
    if v["name"].casefold() in v["password"].casefold():
      raise ValueError("Password cannot contain name")
    if not VALID_PASSWORD_REGEX.match(v["password"]):
      raise ValueError(
          "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 digital"
      )
    v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
    return v


/tmp/ipython-input-511/843481947.py:17: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  role: Role = Field(


Nessa parte, temos a criação da função validate que irá fazer a validação dos dados. Caso os dados estiverem certos, a função exibe esses dados, o que significa que os dados passados estão de acordo com a estrura da classe User. No entanto, se os dados estiverem errados, a função exibe uma mensagem dizendo "User is invalid" e indica os erros na validação. Por fim, temos a criação da função main, que serve apenas para controlar toda a operação, chamando a função validade e passando os casos de teste que foram definidos no dicionário test_data.

In [18]:
def validate(data: dict[str, Any]) -> None:
  try:
    user = User.model_validate(data)
    print(user)
  except ValidationError as e:
    print("User is invalid")
    print(e)


def main() -> None:
  test_data = dict(
    good_data = {
        "name": "Arjan",
        "email": "example@rjancodes.com",
        "password": "Passaword123",
        "role": "Admin",
    },
    bad_role = {
        "name": "Arjan",
        "email": "example@rjancodes.com",
        "password": "Password123",
        "role":"Programmer",
    },
    bad_data = {
        "name": "Arjan",
        "email": "bad email",
        "password": "bad password",
    },
    bad_name = {
        "name": "Arjan<-_->",
        "email": "example@rjancodes.com",
        "password": "Password123",
    },
    duplicate = {
        "name": "Arjan",
        "email": "example@rjancodes.com",
        "password": "Arjan123",
    },
    missing_data = {
        "name": "<bad data>",
        "password": "<bad data>",
    },
  )

  for example_name, data in test_data.items():
    print(example_name)
    validate(data)
    print()

if __name__ == "__main__":
  main()

good_data
name='Arjan' email='example@rjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

bad_data
User is invalid
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 digital [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

bad_name
User is invalid
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://err